# Imports & Setup

In [1]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')  
logging.getLogger('matplotlib').setLevel(logging.WARNING)

# Function definitions

In [2]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

# Main algorithm

In [3]:
import gym
from gym import spaces
# from gym.envs.classic_control import utils
# from gym.error import DependencyNotInstalled
import numpy as np


class Optiver(gym.Env):
    
    def __init__(self):
#         self.max_speed = 8
#         self.max_torque = 2.0
#         self.dt = 0.05
#         self.g = g
#         self.m = 1.0
#         self.l = 1.0

#         self.render_mode = render_mode

#         self.screen_dim = 500
#         self.screen = None
#         self.clock = None
#         self.isopen = True

#         high = np.array([1.0, 1.0, self.max_speed], dtype=np.float32)
    
#         self.action_space = spaces.Box(
#             low=-self.max_torque, high=self.max_torque, shape=(1,), dtype=np.float32
#         )
#         self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        self.min_price = 0
        self.max_price = 200
        self.min_volume = 0
        self.max_volume = 100
        self.min_news = -1
        self.max_news = 1
        self.min_inventory = -100
        self.max_inventory = 100
        
        self.instrument1 = ''
        self.instrument2 = ''
        
        self.exchange = Exchange()
        self.exchange.connect()
        
        self.pnl_last = self.exchange.get_pnl()
        
        self.instrument_list = ['CSCO','NVDA', 'SAN', 'ING', 'PFE']
        self.POSITION_LIMIT = 100
        
        self.instrument_id = 0
        
        high = np.array([self.max_price, self.max_volume, self.max_price, self.max_volume, self.max_price, self.max_volume, self.max_price, self.max_volume, self.max_news, self.max_inventory, self.max_inventory], dtype=np.float32)
        low = np.array([self.min_price, self.min_volume, self.min_price, self.min_volume, self.min_price, self.min_volume, self.min_price, self.min_volume, self.min_news, self.min_inventory, self.min_inventory], dtype=np.float32)
        
        # Obs: [min_ask1, ask_volume1, max_bid1, bid_volume1, min_ask2, ask_volume2, max_bid2, bid_volume2, news, inventory1, inventory2]
        self.observation_space = spaces.Box(low = low, high = high, dtype=np.float32)
        
        self.a_min_price = 0
        self.a_max_price = 1
        self.a_min_volume = 0
        self.a_max_volume = 1
        
        # Action: [Ask_Price_1, Ask_Volume_1, Bid_Price_1, Bid_Volume_1, Ask_Price_2, Ask_Volume_2, Bid_Price_2, Bid_Volume_2]
        high = np.array([self.a_max_price, self.a_max_volume, self.a_max_price, self.a_max_volume, self.a_max_price, self.a_max_volume, self.a_max_price, self.a_max_volume],dtype=np.float32)
        low = np.array([self.a_min_price, self.a_min_volume, self.a_min_price, self.a_min_volume, self.a_min_price, self.a_min_volume, self.a_min_price, self.a_min_volume], dtype=np.float32)
        
        self.action_space = spaces.Box(low = low, high = high, dtype=np.float32)
        

    def step(self, a):
        # a:[Ask_Price_1, Ask_Volume_1, Bid_Price_1, Bid_Volume_1, Ask_Price_2, Ask_Volume_2, Bid_Price_2, Bid_Volume_2]
    
        self.reset()
    
        Ask_Price_1, Ask_Volume_1, Bid_Price_1, Bid_Volume_1, Ask_Price_2, Ask_Volume_2, Bid_Price_2, Bid_Volume_2 = a
        
        minask1, _, maxbid1, _ , minask2, _, maxbid2, _,_,position1,position2 = self.state
        
        price1 = maxbid1 + (minask1 - maxbid1)/2
        price2 = maxbid2 + (minask2 - maxbid2)/2
        
        Ask_Volume_1 = int(Ask_Volume_1*20)
        Ask_Volume_2 = int(Ask_Volume_2*20)
        Bid_Volume_1 = int(Bid_Volume_1*20)
        Bid_Volume_2 = int(Bid_Volume_2*20)
        
        # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
        max_volume_to_buy1 = self.POSITION_LIMIT - position1
        max_volume_to_sell1 = self.POSITION_LIMIT + position1
        
        max_volume_to_buy2 = self.POSITION_LIMIT - position2
        max_volume_to_sell2 = self.POSITION_LIMIT + position2
        
        Ask_Volume_1 = int(min(max_volume_to_sell1,Ask_Volume_1))
        Ask_Volume_2 = int(min(max_volume_to_sell2,Ask_Volume_2))
        Bid_Volume_1 = int(min(max_volume_to_buy1,Bid_Volume_1))
        Bid_Volume_2 = int(min(max_volume_to_buy2,Bid_Volume_2))
        
        Bid_Price_1 = round_down_to_tick(-Bid_Price_1 + price1, 0.1)
        Bid_Price_2 = round_down_to_tick(-Bid_Price_2 + price2, 0.1)
        Ask_Price_1 = round_up_to_tick(Ask_Price_1 + price1, 0.1)
        Ask_Price_2 = round_up_to_tick(Ask_Price_2 + price2, 0.1)
        
        self.exchange.delete_orders(self.instrument1)
        self.exchange.delete_orders(self.instrument2)
        
        penalty = 0
        
        if (Ask_Price_1 - Bid_Price_1) > 0.1 :
            insert_quotes(self.exchange, self.exchange.get_instruments()[self.instrument1], Bid_Price_1, Ask_Price_1, Bid_Volume_1, Ask_Volume_1)
        else:
            penalty+= -4
        if  (Ask_Price_2 - Bid_Price_2) > 0.1:    
            insert_quotes(self.exchange, self.exchange.get_instruments()[self.instrument2], Bid_Price_2, Ask_Price_2, Bid_Volume_2, Ask_Volume_2)
        else: penalty+= -4
        
        print(f'{self.instrument1}: {[Bid_Price_1, Ask_Price_1, Bid_Volume_1, Ask_Volume_1]}')
        # print([Bid_Price_1, Ask_Price_1, Bid_Volume_1, Ask_Volume_1])
        print(f'{self.instrument2}: {[Bid_Price_2, Ask_Price_2, Bid_Volume_2, Ask_Volume_2]}')
        # print([Bid_Price_2, Ask_Price_2, Bid_Volume_2, Ask_Volume_2])
        
        # time.sleep(1)
        
        pnl_current = self.exchange.get_pnl()
        pnl_dif = pnl_current - self.pnl_last
        self.pnl_last = pnl_current
        
        return self._get_obs(), pnl_dif + penalty, False, {}

    def reset(self):
        # super().reset()
#         if options is None:
#             high = np.array([DEFAULT_X, DEFAULT_Y])
#         else:
#             # Note that if you use custom reset bounds, it may lead to out-of-bound
#             # state/observations.
#             x = options.get("x_init") if "x_init" in options else DEFAULT_X
#             y = options.get("y_init") if "y_init" in options else DEFAULT_Y
#             x = utils.verify_number_and_cast(x)
#             y = utils.verify_number_and_cast(y)
#             high = np.array([x, y])
#         low = -high  # We enforce symmetric limits.
#         self.state = self.np_random.uniform(low=low, high=high)
#         self.last_u = None

#         if self.render_mode == "human":
#             self.render()
#         return self._get_obs(), {}
        
        # Obs: [min_ask1, ask_volume1, max_bid1, bid_volume1, min_ask2, ask_volume2, max_bid2, bid_volume2, news, inventory1, inventory2]
        
        self.instrument_id = (self.instrument_id + 1) % 5
        
        self.instrument1 = self.instrument_list[self.instrument_id]
        self.instrument2 = self.instrument1 + '_B'
        instrument_order_book1 = self.exchange.get_last_price_book(self.instrument1)
        instrument_order_book2 = self.exchange.get_last_price_book(self.instrument2)
        
        while not instrument_order_book1 or not instrument_order_book2 or not instrument_order_book1.asks or not instrument_order_book2.asks or not instrument_order_book1.bids or not instrument_order_book2.bids:
            self.instrument_id = (self.instrument_id + 1) % 5
            self.instrument1 = self.instrument_list[self.instrument_id]
            self.instrument2 = self.instrument1 + '_B'
            instrument_order_book1 = self.exchange.get_last_price_book(self.instrument1)
            instrument_order_book2 = self.exchange.get_last_price_book(self.instrument2)
            # print(instrument_order_book1)
            # print(self.instrument1)
            # print('fail')
        
        # print(instrument_order_book1)
        # print(instrument_order_book2)
        
        min_ask1 = instrument_order_book1.asks[0].price
        ask_volume1 = instrument_order_book1.asks[0].volume
        
        max_bid1 = instrument_order_book1.bids[0].price
        bid_volume1 = instrument_order_book1.bids[0].volume
        
        min_ask2 = instrument_order_book2.asks[0].price
        ask_volume2 = instrument_order_book2.asks[0].volume
        
        max_bid2 = instrument_order_book2.bids[0].price
        bid_volume2 = instrument_order_book2.bids[0].volume
        
        news = 0
        
        inventory1 = self.exchange.get_positions()[self.instrument1]
        inventory2 = self.exchange.get_positions()[self.instrument2]
        
        self.state = np.array([min_ask1, ask_volume1, max_bid1, bid_volume1, min_ask2, ask_volume2, max_bid2, bid_volume2, news, inventory1, inventory2], dtype=np.float32)
        
        return self._get_obs()

    def _get_obs(self):
        # print(self.state)
        return self.state

In [4]:
import gym
import stable_baselines3 as sb3
from stable_baselines3.common.env_util import make_vec_env

env = Optiver()

# vec_env = make_vec_env(lambda: env, n_envs=1)

# model = sb3.PPO("MlpPolicy", env)
model = sb3.PPO.load('./model2.zip',env)

2023-11-19 08:30:33,762 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [ ]:
# import gym
# import stable_baselines3 as sb3
# from stable_baselines3.common.env_util import make_vec_env
# env = Optiver()

# vec_env = make_vec_env(lambda: env, n_envs=1)

# model = sb3.PPO("MlpPolicy", vec_env)

model.learn(total_timesteps=10000000000000)

# while True:

#     for instrument in instruments:
#         obs, info = env.reset(instrument)
#         action, _states = model.predict(obs)
#         obs, reward, terminated, truncated, info = env.step(action)

# env.close()

SAN: [54.800000000000004, 55.900000000000006, 0, 0]
SAN_B: [55.800000000000004, 56.900000000000006, 0, 2]
ING: [54.0, 54.400000000000006, 0, 6]
ING_B: [54.0, 54.400000000000006, 5, 1]
PFE: [40.1, 40.800000000000004, 20, 18]
PFE_B: [40.300000000000004, 40.400000000000006, 0, 20]
CSCO: [44.7, 45.2, 20, 0]
CSCO_B: [45.1, 45.800000000000004, 0, 20]
NVDA: [66.4, 66.5, 0, 0]
NVDA_B: [66.10000000000001, 67.10000000000001, 20, 4]
SAN: [55.800000000000004, 55.900000000000006, 8, 10]
SAN_B: [54.800000000000004, 56.0, 20, 0]
ING: [54.300000000000004, 55.400000000000006, 5, 20]
ING_B: [54.300000000000004, 54.400000000000006, 6, 0]
PFE: [39.6, 40.800000000000004, 0, 0]
PFE_B: [40.5, 41.6, 3, 20]
CSCO: [44.900000000000006, 45.1, 4, 0]
CSCO_B: [45.1, 45.800000000000004, 7, 0]
NVDA: [64.7, 65.8, 10, 4]
NVDA_B: [66.2, 66.4, 0, 0]
SAN: [54.7, 55.800000000000004, 11, 20]
SAN_B: [55.7, 55.900000000000006, 0, 7]
ING: [53.300000000000004, 54.400000000000006, 14, 0]
ING_B: [54.300000000000004, 54.40000000000

2023-11-19 08:32:52,544 [client    ] [Thread-4    ] Notification [limit-checker]: You breached the risk limits. Your positions have been reduced automatically to bring them within limits.
Automatically reducing your positions comes at a small cost. Your positions are reduced to 90% of the limit at a premium of 2.0% compared to the fair price of the instrument.
The following limits were breached:
[per_underlying_hedged]
instrument=SAN limit_name=delta limit_value=50, your_value=53.0
instrument=ING limit_name=delta limit_value=50, your_value=-58.07242339832868


PFE: [40.300000000000004, 41.5, 19, 0]
PFE_B: [39.400000000000006, 41.400000000000006, 0, 5]
CSCO: [44.6, 44.800000000000004, 0, 0]
CSCO_B: [44.6, 44.7, 0, 0]
NVDA: [64.4, 64.9, 12, 3]
NVDA_B: [64.0, 64.5, 0, 0]
SAN: [55.0, 56.300000000000004, 0, 0]
SAN_B: [54.2, 55.300000000000004, 3, 0]
ING: [52.800000000000004, 54.800000000000004, 0, 0]
ING_B: [52.800000000000004, 54.2, 0, 20]
PFE: [40.400000000000006, 40.5, 20, 16]
PFE_B: [40.300000000000004, 40.400000000000006, 0, 0]
CSCO: [44.400000000000006, 44.7, 11, 0]
CSCO_B: [44.6, 45.5, 6, 20]
NVDA: [64.4, 65.5, 0, 17]
NVDA_B: [64.4, 64.5, 0, 20]
SAN: [55.300000000000004, 55.400000000000006, 0, 0]
SAN_B: [54.2, 56.300000000000004, 10, 0]
ING: [53.800000000000004, 54.5, 6, 20]
ING_B: [53.6, 53.900000000000006, 18, 0]
PFE: [39.6, 40.5, 10, 0]
PFE_B: [40.300000000000004, 40.400000000000006, 17, 0]
CSCO: [44.800000000000004, 44.900000000000006, 1, 0]
CSCO_B: [44.6, 44.7, 3, 2]
NVDA: [63.900000000000006, 65.2, 0, 10]
NVDA_B: [63.800000000000004,

2023-11-19 08:33:59,386 [client    ] [Thread-4    ] Notification [limit-checker]: You breached the risk limits. Your positions have been reduced automatically to bring them within limits.
Automatically reducing your positions comes at a small cost. Your positions are reduced to 90% of the limit at a premium of 2.0% compared to the fair price of the instrument.
The following limits were breached:
[per_underlying_hedged]
instrument=SAN limit_name=delta limit_value=50, your_value=60.99009900990099


SAN: [55.5, 56.2, 8, 0]
SAN_B: [55.6, 56.5, 0, 8]
ING: [52.0, 52.1, 0, 0]
ING_B: [52.0, 52.1, 14, 0]
PFE: [39.7, 40.900000000000006, 0, 18]
PFE_B: [40.1, 40.2, 20, 0]
CSCO: [44.800000000000004, 44.900000000000006, 20, 16]
CSCO_B: [44.400000000000006, 44.900000000000006, 0, 0]


...

ING: [49.300000000000004, 50.900000000000006, 0, 20]
ING_B: [50.400000000000006, 51.5, 20, 1]
PFE: [39.900000000000006, 40.400000000000006, 0, 0]
PFE_B: [39.400000000000006, 41.300000000000004, 1, 0]
CSCO: [43.800000000000004, 44.0, 20, 0]
CSCO_B: [42.900000000000006, 44.0, 0, 0]
NVDA: [62.1, 63.2, 0, 0]
NVDA_B: [63.1, 63.2, 1, 20]
SAN: [56.0, 57.800000000000004, 0, 6]
SAN_B: [57.1, 57.2, 0, 0]
ING: [49.300000000000004, 50.400000000000006, 0, 18]
ING_B: [49.400000000000006, 50.5, 17, 0]
PFE: [40.1, 40.400000000000006, 9, 0]
PFE_B: [39.5, 40.900000000000006, 0, 0]
CSCO: [43.400000000000006, 43.900000000000006, 0, 0]
CSCO_B: [42.900000000000006, 44.0, 17, 8]
NVDA: [63.1, 63.2, 11, 20]
NVDA_B: [63.0, 63.2, 7, 8]
SAN: [56.300000000000004, 57.5, 20, 0]
SAN_B: [56.800000000000004, 57.2, 0, 0]
ING: [49.400000000000006, 50.7, 9, 20]
ING_B: [50.2, 50.5, 0, 0]
PFE: [40.400000000000006, 40.5, 0, 0]
PFE_B: [39.400000000000006, 40.5, 0, 0]
CSCO: [42.800000000000004, 44.1, 20, 0]
CSCO_B: [43.4000000

2023-11-19 08:44:20,248 [client    ] [Thread-4    ] Notification [limit-checker]: You breached the risk limits. Your positions have been reduced automatically to bring them within limits.
Automatically reducing your positions comes at a small cost. Your positions are reduced to 90% of the limit at a premium of 2.0% compared to the fair price of the instrument.
The following limits were breached:
[per_underlying_hedged]
instrument=SAN limit_name=delta limit_value=50, your_value=64.93865030674846
instrument=ING limit_name=delta limit_value=50, your_value=83.97222222222223


NVDA: [62.1, 63.300000000000004, 0, 9]
NVDA_B: [61.5, 62.800000000000004, 2, 18]
SAN: [57.0, 57.5, 0, 0]
SAN_B: [56.2, 57.2, 19, 0]
ING: [50.300000000000004, 51.400000000000006, 4, 11]
ING_B: [49.900000000000006, 51.400000000000006, 0, 0]
PFE: [40.1, 40.2, 0, 0]
PFE_B: [40.1, 41.2, 0, 0]


2023-11-19 08:44:21,366 [client    ] [Thread-4    ] Notification [limit-checker]: You breached the risk limits. Your positions have been reduced automatically to bring them within limits.
Automatically reducing your positions comes at a small cost. Your positions are reduced to 90% of the limit at a premium of 2.0% compared to the fair price of the instrument.
The following limits were breached:
[per_underlying_hedged]
instrument=SAN limit_name=delta limit_value=50, your_value=64.96056091148115


CSCO: [43.900000000000006, 44.0, 0, 11]
CSCO_B: [43.800000000000004, 44.6, 6, 7]
NVDA: [61.800000000000004, 62.7, 7, 0]
NVDA_B: [62.5, 62.6, 0, 0]
SAN: [56.5, 58.0, 0, 0]
SAN_B: [56.800000000000004, 57.2, 0, 20]
ING: [49.300000000000004, 50.400000000000006, 5, 18]
ING_B: [49.6, 51.2, 0, 10]
PFE: [40.1, 41.2, 0, 2]
PFE_B: [39.900000000000006, 40.5, 0, 8]
CSCO: [43.900000000000006, 44.5, 19, 0]
CSCO_B: [43.900000000000006, 44.0, 0, 20]
NVDA: [62.5, 62.7, 0, 0]
NVDA_B: [62.5, 63.5, 0, 20]
SAN: [57.1, 58.2, 2, 0]
SAN_B: [57.1, 57.800000000000004, 2, 20]
ING: [51.1, 52.5, 9, 12]
ING_B: [50.900000000000006, 52.900000000000006, 0, 0]
PFE: [39.400000000000006, 40.2, 20, 0]
PFE_B: [40.1, 40.2, 15, 20]
CSCO: [43.1, 43.900000000000006, 0, 0]
CSCO_B: [43.2, 44.0, 4, 0]
NVDA: [62.5, 62.5, 0, 12]
NVDA_B: [61.5, 62.6, 20, 0]
SAN: [57.1, 57.5, 0, 0]
SAN_B: [56.1, 58.2, 8, 0]
ING: [51.400000000000006, 53.300000000000004, 7, 15]
ING_B: [52.0, 52.900000000000006, 4, 20]
PFE: [40.2, 41.300000000000004, 14

In [6]:
model.save('./model2.zip')

In [8]:
obs = env.reset()
# print(obs)
action, _states = model.predict(obs)
print(action)
# print(env.pnl_last)
# print_positions_and_pnl(env.exchange)

obs, reward, terminated, info = env.step(action)
print(env.exchange.get_last_price_book(env.instrument1))
# print(reward)
# print(env.exchange.get_pnl())

# print_positions_and_pnl(env.exchange)

# for instrument in instruments:
#     obs, info = env.reset(instrument)
#     action, _states = model.predict(obs)
#     obs, reward, terminated, truncated, info = env.step(action)

2023-11-19 02:23:10,758 [client    ] [MainThread  ] Unable to connect to the exchange
2023-11-19 02:23:10,759 [client    ] [MainThread  ] 
Unable to authenticate with the server. Please double-check that your username and password are correct and that you are whitelisted on an instance.
 


Exception: Unable to connect to the exchange

In [4]:
!pip install tensorboard

distutils: /usr/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/include/python3.6/UNKNOWN
sysconfig: /usr/include/python3.6m/UNKNOWN
user = False
home = None
root = None
prefix = None
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.9 MB 24.7 MB/s            
     |████████████████████████████████| 289 kB 67.4 MB/s            
     |████████████████████████████████| 126 kB 49.7 MB/s            
     |████████████████████████████████| 4.6 MB 53.5 MB/s            
     |████████████████████████████████| 781 kB 46.4 MB/s            
     |████████████████████████████████| 181 kB 19.0 MB/s            
     |████████████████████████████████| 97 kB 8.0 MB/s             
     |████████████████████████████████| 4.9 MB 47.4 MB/s            
     |████████████████████████████████| 1.1 MB 56.9 MB/s            
     |████████████████████████████████| 181 kB 39.3 MB/s            
     |███